# AutoGluon Time Series - Forecasting In-depth
In this notebook we will discuss the advanced functionality of AutoGluon's `timeseries` module.

We will use these features to build a demand forecasting model for grocery sales.

This tutorial assumes that you have already completed the [Forecasting Essentials](01-forecasting-essentials.ipynb) notebook.

In [ ]:
!pip install -q uv
# Use CPU version of PyTorch for faster installation
!uv pip install torch==2.5 torchvision --index-url https://download.pytorch.org/whl/cpu
!uv pip install autogluon==1.2

In [ ]:
import matplotlib.pyplot as plt  # for plotting only
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

## Forecasting time series with additional information
In real-world forecasting problems we often have access to additional information, beyond just the historic time series values.
AutoGluon supports two types of such additional information: static features and time-varying covariates.

### Static features
Static features are the time-independent attributes (metadata) of a time series.
These may include information such as:

- location, where the time series was recorded (country, state, city)
- fixed properties of a product (brand name, color, size, weight)
- store ID or product ID

Providing this information may, for instance, help forecasting models generate similar demand forecasts for stores located in the same city.

### Time-varying covariates
Covariates are the time-varying features that may influence the target time series.
They are sometimes also referred to as dynamic features, exogenous regressors, or related time series.
AutoGluon supports two types of covariates:

- *known* covariates that are known for the entire forecast horizon, such as
    - holidays
    - day of the week, month, year
    - promotions

- *past* covariates that are only known up to the start of the forecast horizon, such as
    - sales of other products
    - temperature, precipitation
    - transformed target time series


![Target time series with one past covariate and one known covariate.](https://autogluon-timeseries-datasets.s3.us-west-2.amazonaws.com/public/figures/forecasting-indepth5.png)

In AutoGluon, both `known_covariates` and `past_covariates` are stored as additional columns in the `TimeSeriesDataFrame`.


### Example: Grocery sales data
For example, let's have a look at the grocery sales dataset. The dataset contains weekly sales data for various food items sold at different stores.


In [ ]:
df = pd.read_csv("https://autogluon.s3.us-west-2.amazonaws.com/datasets/timeseries/grocery_sales/test.csv")
df

As usual, `item_id` contains the unique identifier of each time series and `timestamp` contains the timestamps of the observations.
The `unit_sales` column is the target time series that we want to forecast.

We will use the remaining columns as the time-varying covariates:
- `scaled_price` - scaled price of each product on the given week
- `promotion_email`, `promotion_homepage` - binary indicators that show whether the product was promoted in an email / on the homepage during the given week

In [ ]:
df.describe().round(2)

Now we load an additional table containing the static (time-independent) features of the time series.

In [ ]:
static_df = pd.read_csv("https://autogluon.s3.us-west-2.amazonaws.com/datasets/timeseries/grocery_sales/static.csv")
static_df

This table contains the time-independent attributes of each time series, indicating the type of the product and the location code of the store where it is sold.

------

We need to convert the data into a `TimeSeriesDataFrame` to use it in AutoGluon.

In [ ]:
full_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp",
    static_features_df=static_df,
)

The `TimeSeriesDataFrame` that we created now holds both the dynamic and the static features of the dataset.

In [ ]:
full_data.head()

In [ ]:
full_data.static_features.head()

### Train-test split
Currently, `full_data` contains all the available historic sales data. We will use the last 4 weeks of observations as a **test set** (not used during training) to measure the performance of the trained predictor.

This is very important to get a fair estimate of the performance. If we compute the scores on the same data as we used for training, our results would be overly optimistic - the predictor could have "memorized" the training data in a way that does not generalize into the future.

In [ ]:
prediction_length = 4  # we will forecast 4 weeks into the future
train_data, test_data = full_data.train_test_split(prediction_length=prediction_length)

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    target="unit_sales",
)
predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=120,
)

Now, we evaluate the trained models on the held-out test data.

In [ ]:
predictor.leaderboard(test_data)

Let's have a closer look at the predictor logs to understand how it interpreted the different features.
```
...
Provided data contains following columns:
	target: 'unit_sales'
	past_covariates:
		categorical:        []
		continuous (float): ['scaled_price', 'promotion_email', 'promotion_homepage']
	static_features:
		categorical:        ['product_category', 'product_subcategory']
		continuous (float): ['product_code', 'location_code']
...
```
We notice a few problems here.

First, the columns `'scaled_price'`, `'promotion_email'` and `'promotion_homepage'` were used as past covariates, even though we know these columns in advance (we decide how to set the price or whether to run a promotion). 

To fix this problem, we need to explicitly state that these columns are known using the `known_covariates_names` argument when creating the predictor.

In [ ]:
predictor_fixed = TimeSeriesPredictor(
    prediction_length=prediction_length,
    target="unit_sales",
    known_covariates_names=["scaled_price", "promotion_email", "promotion_homepage"],
)

The second problem is that the static features `product_code` and `location_code` (originally of integer type) are interpreted as continuous attributes, even though they are meant to be used as categorical features.

To fix this problem, we need to explicitly set their type to `"category"`.

In [ ]:
train_data.static_features["product_code"] = train_data.static_features["product_code"].astype("category")
train_data.static_features["location_code"] = train_data.static_features["location_code"].astype("category")

Now let's train the new predictor after we fixed the data issues.

In [ ]:
predictor_fixed.fit(train_data, presets="medium_quality", time_limit=120)

In [ ]:
predictor_fixed.leaderboard(test_data)

### Predicting with known covariates
When creating the predictor, we said that columns listed under `known_covariates` will be known in the future.
Therefore, when we call `predict`, we need to provide the future values of these columns.

The `known_covariates` must contain information for the `prediction_length` following the last observations in `train_data`

In [ ]:
train_data.tail()

We can get this information by selecting the necessary columns from the test data that we set aside before.

In [ ]:
future_data = test_data.slice_by_timestep(-prediction_length, None)  # select the last prediction_length values of each time series
known_covariates = future_data.drop(columns=[predictor.target])
known_covariates.head()

In [ ]:
predictions = predictor.predict(train_data, known_covariates=known_covariates)
predictions.head()

### Feature importance

We see that the new predictor with `known_covariates` achieves a much better test score than before!

This means that the features are quite important for improving the forecast accuracy. But which ones are the most important here? 

We can answer this question using the `feature_importance()` method.

In [ ]:
feat_importance = predictor_fixed.feature_importance(test_data)
feat_importance.sort_values(by="importance", ascending=False)

We see that `scaled_price` and `product_code` are the most informative features in this dataset, and removing them would result in a significant drop in accuracy.

For more information on how the feature importance algorithm works in AutoGluon, check out the [documentation page](https://auto.gluon.ai/stable/api/autogluon.timeseries.TimeSeriesPredictor.feature_importance.html).

## Challenge: Investigate the effect of the `eval_metric`
Now let's put our knowledge to practice to better understand how AutoGluon works.

In this challenge, you need to complete the following steps:
1. Train two predictors
    - one with `eval_metric="SQL"` ([scaled quantile loss](https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-metrics.html#autogluon.timeseries.metrics.SQL)), an evaluation metric for probabilistic forecasts
    - another with `eval_metric="RMSE"` ([root mean squared error](https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-metrics.html#autogluon.timeseries.metrics.RMSE)), an evaluation metric for point forecasts
2. Compute the `SQL` and `RMSE` metrics on the `test_data` for both predictors.

In [ ]:
# Your code here

If you don't feel like writing the code yourself, click on the next section to reveal one possible implementation.

### Challenge solution

In [ ]:
metrics = ["SQL", "RMSE"]
predictor_for_metric = {}
for metric in metrics:
    predictor_for_metric[metric] = TimeSeriesPredictor(
        prediction_length=prediction_length,
        target="unit_sales",
        known_covariates_names=["scaled_price", "promotion_email", "promotion_homepage"],
        eval_metric=metric,
        path=f"predictor-{metric}",
    ).fit(train_data, presets="medium_quality", time_limit=120)

In [ ]:
for metric in metrics:
    scores = predictor_for_metric[metric].evaluate(test_data, metrics=metrics)
    print(f"Predictor trained with `eval_metric='{metric}'`: {scores}")

### Discussion
We can observe that the predictor trained with the `SQL` metric achieves better `SQL` on the test set. Similarly, the predictor trained with `RMSE` achieves better `RMSE` on the test set.

When fitting `WeightedEnsemble`, AutoGluon looks for the combination of models that optimizes the chosen `eval_metric`. Not surprisingly, different models are selected when different metrics are used to measure accuracy. The resulting ensembles are tailored specifically to the metrics that they were trained with.


## Advanced configuration options

### Basic configuration with `presets` and `time_limit`
We can fit `TimeSeriesPredictor` with different pre-defined configurations using the `presets` argument of the `fit` method.

```python
predictor = TimeSeriesPredictor(...)
predictor.fit(train_data, presets="medium_quality")
```

Higher quality presets usually result in better forecasts but take longer to train.
The following presets are available:

| Preset         | Description                                          | Use Cases                                                                                                                                               | Fit Time (Ideal) | 
| :------------- | :----------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------ | :--------------- | 
| `fast_training`  | Fit simple statistical and baseline models + fast tree-based models   | Fast to train but may not be very accurate   |  0.5x |
| `medium_quality` | Same models as in `fast_training` + deep learning model `TemporalFusionTransformer`           | Good forecasts with reasonable training time         | 1x             |
| `high_quality`   | More powerful deep learning, machine learning, and statistical forecasting models   | Much more accurate than ``medium_quality``, but takes longer to train | 3x |
| `best_quality`   | Same models as in `high_quality`, more cross-validation windows | Typically more accurate than `high_quality`, especially for datasets with few (<50) time series | 6x             |

You can find more information about the [presets](https://github.com/autogluon/autogluon/blob/stable/timeseries/src/autogluon/timeseries/configs/presets_configs.py) and the [models includes in each preset](https://github.com/autogluon/autogluon/blob/stable/timeseries/src/autogluon/timeseries/models/presets.py#L109) in the AutoGluon source code.

Another way to control the training time is using the `time_limit` argument.

```python
predictor.fit(
    train_data,
    time_limit=60 * 60,  # total training time in seconds
)
```

If no `time_limit` is provided, the predictor will train until all models have been fit.


### Manually configuring models
Advanced users can override the presets and manually specify what models should be trained by the predictor using the `hyperparameters` argument.

```python
predictor = TimeSeriesPredictor(...)

predictor.fit(
    ...
    hyperparameters={
        "DeepAR": {},
        "Theta": [
            {"decomposition_type": "additive"},
            {"seasonal_period": 1},
        ],
    }
)
```

The above example will train three models:

* ``DeepAR`` with default hyperparameters
* ``Theta`` with additive seasonal decomposition (all other parameters set to their defaults)
* ``Theta`` with seasonality disabled (all other parameters set to their defaults)

You can also exclude certain models from the presets using the `excluded_model_type` argument.
```python
predictor.fit(
    ...
    presets="high_quality",
    excluded_model_types=["AutoETS", "AutoARIMA"],
)
```

For the full list of available models and the respective hyperparameters, see [Forecasting Model Zoo](https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-model-zoo.html).

## Chronos - a foundation model for time series forecsating
AutoGluon-TimeSeries (AG-TS) includes [Chronos](https://github.com/amazon-science/chronos-forecasting) family of forecasting models. Chronos models are pretrained on a large collection of real & synthetic time series data, which enables them to make accurate forecasts on new data out of the box.

AG-TS provides a robust and easy way to use Chronos through the familiar `TimeSeriesPredictor` API.

### Getting started with Chronos

Being a pretrained model for zero-shot forecasting, Chronos is different from other models available in AG-TS. 
Specifically, Chronos models do not really `fit` time series data. However, when `predict` is called, they carry out a relatively more expensive computation that scales linearly with the number of time series in the dataset. In this aspect, they behave like local statistical models such as ETS or ARIMA, where all computation happens during inference. 

AutoGluon supports both the original Chronos models (e.g., [`chronos-t5-large`](https://huggingface.co/autogluon/chronos-t5-large)), as well as the new, more accurate and up to 250x faster Chronos-Bolt⚡ models (e.g., [`chronos-bolt-base`](https://huggingface.co/autogluon/chronos-bolt-base)). 

The easiest way to get started with Chronos is through the model-specific presets. 

- **(recommended)** The new, fast Chronos-Bolt️ models can be accessed using the `"bolt_tiny"`, `"bolt_mini"`, `"bolt_small"` and `"bolt_base"` presets.
- The original Chronos models can be accessed using the `"chronos_tiny"`, `"chronos_mini"`, `"chronos_small"`, `"chronos_base"` and `"chronos_large"` presets.

Note that the original Chronos models of size `small` and above require a GPU to run, while all Chronos-Bolt models can be run both on a CPU and a GPU.

Alternatively, Chronos can be combined with other time series models using presets `"medium_quality"`, `"high_quality"` and `"best_quality"`. More details about these presets are available in the documentation for [`TimeSeriesPredictor.fit`](https://auto.gluon.ai/stable/api/autogluon.timeseries.TimeSeriesPredictor.fit.html).

### Zero-shot forecasting with Chronos
We create the [TimeSeriesPredictor](https://auto.gluon.ai/stable/api/autogluon.timeseries.TimeSeriesPredictor.html) and select the `"bolt_small"` presets to use the Chronos-Bolt (Small, 48M) model in zero-shot mode.

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    target="unit_sales",
).fit(
    train_data,
    presets="bolt_small",
)

As promised, Chronos does not take any time to `fit`. The `fit` call merely serves as a proxy for the `TimeSeriesPredictor` to do some of its chores under the hood, such as inferring the frequency of time series and saving the predictor's state to disk. 

Let's use the `predict` method to generate forecasts, and the `plot` method to visualize them.

In [ ]:
predictions = predictor.predict(train_data)
predictor.plot(
    data=test_data,
    predictions=predictions,
    item_ids=test_data.item_ids[:2],  # plot first two items
);
plt.show()

In the above example we used Chronos for zero-shot forecasting. In this mode, the pretrained model is used to directly generate predictions on new data out of the box. This is the simplest and fastest way to generate predictions with Chronos.

In several [independent evaluations](https://arxiv.org/abs/2403.07815) we found Chronos to be effective in zero-shot forecasting. 
The accuracy of Chronos-Bolt (base) often exceeds statistical baseline models, and is often comparable to deep learning  models such as `TemporalFusionTransformer` or `PatchTST`.

### Beyond zero-shot forecasting

In addition to zero-shot forecasting, AutoGluon provides advanced features that can improve the accuracy of Chronos models:
- **Fine-tuning** Chronos models on custom data to improve the accuracy
- Handling **covariates & static features** by combining Chronos with a tabular regression model

You can learn more about these features in the [Forecasting with Chronos tutorial](https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html).

## Next Steps

Please continue to the next workshop section **"Train and deploy AutoGluon predictors on SageMaker"** on Workshop Studio.